In [6]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os
import glob
import pydicom
import matplotlib.pyplot as plt
from time import time_ns
import sys
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + "/rsna-2024/")
from rsna_dataloader import create_datasets_and_loaders, clean_training_data, retrieve_training_data
import torch

In [27]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Lambda(lambda x: (x * 255).astype(np.uint8)),  # Convert back to uint8 for PIL
    transforms.ToPILImage(),
    transforms.Resize((512, 512)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])


In [28]:
label_map = {'normal_mild': 0, 'moderate': 1, 'severe': 2}
series_desc = ('Sagittal T1', 'Axial T2', 'Sagittal T2/STIR')
data = retrieve_training_data("/Users/victorsahin/PycharmProjects/pythonProject/data/rsna-2024-lumbar-spine-degenerative-classification/")
data = clean_training_data(data, "/Users/victorsahin/PycharmProjects/pythonProject/data/rsna-2024-lumbar-spine-degenerative-classification/")

loaders = {e : create_datasets_and_loaders(data, e, transform) for e in series_desc}
loaders 

{'Sagittal T1': (<torch.utils.data.dataloader.DataLoader at 0x2a6300f10>,
  15779,
  3945),
 'Axial T2': (<torch.utils.data.dataloader.DataLoader at 0x2a6184c50>,
  15376,
  3844),
 'Sagittal T2/STIR': (<torch.utils.data.dataloader.DataLoader at 0x2a5d42350>,
  7798,
  1950)}

In [34]:
import importlib
import unet
import train_unet
importlib.reload(unet)
importlib.reload(train_unet)
from train_unet import train
from unet import UNet
models = {e : train(loaders[e][0], loaders[e][1], label_map) for e in loaders}

  0%|          | 1/1973 [01:39<54:34:06, 99.62s/it]


KeyboardInterrupt: 